# Convert Tensorflow model checkpoints to saved model

Given the checkpoints exported from Tensorflow model training, our goal is to convert those checkpoints into saved model for inference purpose.<br>
Checkpoints is a binary file which contains all the values of the weights, biases, gradients and all the other variables saved. This file has an extension .ckpt. Checkpoints do not contain any description of the computation defined by the model and thus are typically only useful when source code that will use the saved parameter values is available.<br>
A saved model contains a complete tensorflow program, including trained parameters and computation. It does not require the original model building code to run, which makes it useful for sharing or deploying with TFLite, tensorflow.js, Tensorflow Serving or Tensorflow Hub.

**Note** - We also assume that the script will be used as a Google Colab notebook. But this can be changed according to the needs of users. They can modify this in case they are working on their local workstation, remote server or any other database. This colab notebook can be changed to a regular jupyter notebook running on a local machine according to the need of the users.

## Import libraries & clone the TF model directory

In [ ]:
# install model-garden official and RESTART RUNTIME of the colab
!pip install tf-models-official

In [ ]:
import os
from google.colab import drive
import yaml
import tensorflow as tf

In [ ]:
# use this if your model and data are stored in the google drive
drive.mount('/content/gdrive')

try:
  !ln -s /content/gdrive/My\ Drive/ /mydrive
  print('Successful')
except Exception as e:
  print(e)
  print('Not successful')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
Successful


In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [ ]:
%cd models

/content/models


## **MUST CHANGE** - Define the parameters according to your need

In [ ]:
# this parameter depends on the backbone you will be using. In our 
# case we used resnet backbone
EXPERIMENT_TYPE = 'maskrcnn_resnetfpn_coco' #@param {type:"string"}

# path to the folder where all the files and checkpoints after model training 
# are exported to
CHECKPOINT_PATH = '/mydrive/plastics_model/version_1/' #@param {type:"string"}

# path where the saved model will be exported to
EXPORT_DIR_PATH = '/mydrive/plastics_model/experiment/' #@param {type:"string"}

# config files are always stored with the checkpoints
CONFIG_FILE= CHECKPOINT_PATH + 'params.yaml'

In [ ]:
# config files are always stored with the checkpoints
# read the params.yaml file in order to get the height and width of an image
with open(CONFIG_FILE) as f:
    my_dict = yaml.safe_load(f)

HEIGHT = my_dict['task']['model']['input_size'][0]
WIDTH = my_dict['task']['model']['input_size'][1]
print(HEIGHT, WIDTH)

## calling the function to convert checkpoints to saved model

In [ ]:
# run the conversion command
!python -m official.vision.serving.export_saved_model --experiment=$EXPERIMENT_TYPE \
                   --export_dir=$EXPORT_DIR_PATH \
                   --checkpoint_path=$CHECKPOINT_PATH \
                   --batch_size=1 \
                   --input_image_size=$HEIGHT,$WIDTH \
                   --input_type=tflite \
                   --config_file=$CONFIG_FILE

# Convert saved model to TF Lite model

Given the saved model after Tensorflow model training, our goal is to convert saved model to TFLite for inference purpose on edge devices. 

Tensorflow Lite is a set of tools that enables on-device machine learning by helping developers run their models on mobile, embedded and edge devices.

## **MUST CHANGE** - Define the parameters according to your need

In [ ]:
# path where the tflite model will be written with its name
TFLITE_PATH = '/mydrive/gtech/MRFs/Recykal/Latest_sharing_by_sanket/Google_Recykal/Taxonomy_version_2/model_version_1/plastics_model/tflite_fan/model.tflite' #@param {type:"string"}

# path where saved model parameters are saved
SAVED_MODEL_DIR = EXPORT_DIR_PATH + '/saved_model/'

## conversion of saved model to tflite

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir=SAVED_MODEL_DIR)  
tflite_model = converter.convert() 
with open(TFLITE_PATH, 'wb') as f:
  f.write(tflite_model)